In [1]:
import os
import json
import numpy as np

In [2]:
results_dir = '/mnt/lts4-pathofm/scratch/home/ogut/xMIL/reproducing/results'
datasets = os.listdir(results_dir)

In [3]:
import os
import json
import numpy as np

def compute_mean_std(data, key_path):
    """Compute mean and standard deviation for given metrics using a nested key path."""
    try:
        # Extract the first test run's metrics keys
        first_entry = data[0]
        for key in key_path.split('.'):
            first_entry = first_entry[key]  # Navigate into nested keys

        # Compute mean and std for all runs
        return {
            key: (np.mean([entry[key_path.split('.')[0]][key_path.split('.')[1]][key] for entry in data]), 
                  np.std([entry[key_path.split('.')[0]][key_path.split('.')[1]][key] for entry in data]))
            for key in first_entry
        }
    except KeyError:
        print(f"KeyError: {key_path} not found in data.")
        return {}

results_dir = '/mnt/lts4-pathofm/scratch/home/ogut/xMIL/reproducing/results'
datasets = os.listdir(results_dir)

for dataset in datasets:
    dataset_dir = os.path.join(results_dir, dataset)
    models = os.listdir(dataset_dir)

    for model in models:
        model_dir = os.path.join(dataset_dir, model)
        date_folder = os.listdir(model_dir)[0]
        results_json_dir = os.path.join(model_dir, date_folder)
        results_file_path = os.path.join(results_json_dir, "results.json")

        if model in ["conjmil", "addmil"]:

            with open(results_file_path, 'r') as f:
                data = json.load(f)

            test_auc = [run['test']['auc'] for run in data]
            mean_std_results = {'test_auc': (np.mean(test_auc), np.std(test_auc))}

            mean_std_results.update({
                'attention': compute_mean_std(data, 'test.attention'),
                'patch_scores': compute_mean_std(data, 'test.patch_scores')
            })

        elif model == "attnmil":

            with open(results_file_path, 'r') as f:
                data = json.load(f)

            test_auc = [run['test']['auc'] for run in data]
            mean_std_results = {'test_auc': (np.mean(test_auc), np.std(test_auc))}

            mean_std_results.update({
                'attention': compute_mean_std(data, 'test.attention'),
                'random': compute_mean_std(data, 'test.random'),
                'perturbation_keep': compute_mean_std(data, 'test.perturbation_keep'),
                'grad2': compute_mean_std(data, 'test.grad2'),
                'gi': compute_mean_std(data, 'test.gi'),
                'ig': compute_mean_std(data, 'test.ig'),
                'lrp': compute_mean_std(data, 'test.lrp'),
            })

        # Formatted output
        formatted_output = f"{dataset} and {model}\n"
        formatted_output += f"Test AUC: {mean_std_results['test_auc'][0]:.4f} ± {mean_std_results['test_auc'][1]:.4f}\n\n"

        for key, metrics in mean_std_results.items():
            if key != 'test_auc':
                formatted_output += f"{key.capitalize()} Metrics:\n"
                for metric, (mean, std) in metrics.items():
                    formatted_output += f"  {metric}: {mean:.4f} ± {std:.4f}\n"
                formatted_output += "\n"

        print(formatted_output)


adjacent_pairs and conjmil
Test AUC: 0.7554 ± 0.0440

Attention Metrics:
  auroc_pos: 0.5872 ± 0.0434
  auprc_pos: 0.6710 ± 0.0326
  ndgcn: 0.9508 ± 0.0076
  pearsonr: 0.2227 ± 0.0062
  spearmanr: 0.2242 ± 0.0229
  auroc_2: 0.5000 ± 0.0000
  auprc_2: 0.6028 ± 0.0140

Patch_scores Metrics:
  auroc_pos: 0.7070 ± 0.0298
  auprc_pos: 0.7423 ± 0.0215
  ndgcn: 0.9642 ± 0.0043
  pearsonr: 0.2273 ± 0.0033
  spearmanr: 0.2639 ± 0.0034
  auroc_2: 0.7062 ± 0.0272
  auprc_2: 0.7418 ± 0.0230


adjacent_pairs and attnmil
Test AUC: 0.7983 ± 0.0334

Attention Metrics:
  auroc_pos: 0.6211 ± 0.0254
  auprc_pos: 0.7225 ± 0.0227
  ndgcn: 0.9634 ± 0.0047
  pearsonr: 0.2279 ± 0.0046
  spearmanr: 0.2533 ± 0.0075
  auroc_2: 0.5000 ± 0.0000
  auprc_2: 0.6267 ± 0.0050

Random Metrics:
  auroc_pos: 0.5006 ± 0.0042
  auprc_pos: 0.5424 ± 0.0031
  ndgcn: 0.9153 ± 0.0010
  pearsonr: 0.2504 ± 0.0034
  spearmanr: 0.2516 ± 0.0032
  auroc_2: 0.4985 ± 0.0036
  auprc_2: 0.5410 ± 0.0020

Perturbation_keep Metrics:
  auroc_